In [1]:
from __future__ import division
import time
import nltk
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize, word_tokenize 
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.models import Word2Vec
import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_datasets as tfds
nltk.download('brown')

import multiprocessing
import re

import warnings 
warnings.filterwarnings(action = 'ignore') 

from gensim.scripts.glove2word2vec import glove2word2vec


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


### variable

In [2]:
alpha = 3/4
window_size = 5
fileName = 'dataset.txt'

# text classification
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a context.

In [3]:
# Normalize the string (marks and words are seperated, lower the word,...)
def normalizeString(s):
    # Seperate words and marks by adding spaces between them
    marks = '[.!?,-${}()\":\\;/]'
    r = "(["+"\\".join(marks)+"])"
    s = re.sub(r, r" \1 ", s)
    # replace continuous spaces with a single space
    s = re.sub(r"\s+", r" ", s).strip()
    return s.lower()

In [4]:
class Vocabulary:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0  # Count words in corpus
        self.word2prob = {}

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    
    def generateProb(self):
        for item in self.word2count.items():
            self.word2prob[item[0]] = (item[1] / sum(self.word2count.values())) ** alpha

In [5]:
def read_data(file):
    vocabulary = Vocabulary()
    traindata = []
    sents = open(file, 'r', encoding="utf8").readlines()
    for sent in sents:
      sent = normalizeString(sent)
      vocabulary.addSentence(sent)
      traindata.append(sent.split())
    vocabulary.generateProb()
    return traindata, vocabulary

In [6]:
train, vocabulary = read_data(fileName)
print(train[:2])
print(len(train))

[['thân', 'em', 'như', 'hạt', 'mưa', 'sa'], ['hạt', 'vào', 'đài', 'các', 'hạt', 'ra', 'ruộng', 'cày']]
974


### Sylas embedding

In [7]:

  
#  Reads ‘alice.txt’ file 
sample = open("C:\\Users\\Admin\\Desktop\\alice.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
f = s.replace("\n", " ") 
  
data = [] 
  
# iterate through each sentence in the file 
for i in sent_tokenize(f): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Admin\\Desktop\\alice.txt'

#### Cbow word embedding

In [8]:
# Create CBOW model 
model_cbow = gensim.models.Word2Vec(train, min_count = 1,  
                              size = 100, window = 5) 

In [9]:
print(model_cbow)

Word2Vec(vocab=2175, size=100, alpha=0.025)


#### Skipgram word embedding

In [10]:
model_skipgram = gensim.models.Word2Vec(train, min_count = 1, size = 100, 
                                             window = 5, sg = 1) 

## Glove word embedding 

In [12]:
word2vec_output_file = 'Glove_embedding.txt'
model_glove = glove2word2vec(fileName, word2vec_output_file)
print(model_glove)

(974, 5)


In [13]:
# Print results 
print("Cosine similarity between 'alice' " + 
               "and 'wonderland' - CBOW : ", 
    model_cbow.similarity('thân', 'em')) 
      
print("Cosine similarity between 'alice' " +
                 "and 'machines' - CBOW : ", 
      model_cbow.similarity('thân', 'em')) 
  
# Create Skip Gram model 

  
# Print results 
print("Cosine similarity between 'alice' " +
          "and 'wonderland' - Skip Gram : ", 
    model_skipgram.similarity('thân', 'em')) 
      
print("Cosine similarity between 'alice' " +
            "and 'machines' - Skip Gram : ", 
      model_skipgram.similarity('thân', 'em')) 

Cosine similarity between 'alice' and 'wonderland' - CBOW :  0.9985078
Cosine similarity between 'alice' and 'machines' - CBOW :  0.9985078
Cosine similarity between 'alice' and 'wonderland' - Skip Gram :  0.9995554
Cosine similarity between 'alice' and 'machines' - Skip Gram :  0.9995554


## text classification model

In [14]:
hub_layer = hub.KerasLayer(model_glove, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

TypeError: not all arguments converted during string formatting

In [15]:
model = tf.keras.Sequential()
model.add(model_glove)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

TypeError: The added layer must be an instance of class Layer. Found: (974, 5)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### train model

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

### evaluate model

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))